<a href="https://colab.research.google.com/github/sbtnoob12/BDA-Alcohol/blob/family_variables_prediction/BDA_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
pip install dmba

In [61]:
#import required libraries
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
# from mord import LogisticIT
import matplotlib.pylab as plt
import seaborn as sns
import dmba
from dmba import classificationSummary, gainsChart, liftChart
from dmba.metric import AIC_score
from sklearn import metrics



%matplotlib inline

In [62]:
stud_mat_df = pd.read_csv('/content/student-mat.csv')
stud_mat_df
stud_por_df = pd.read_csv('/content/student-por.csv')
stud_por_df

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,MS,F,19,R,GT3,T,2,3,services,other,...,5,4,2,1,2,5,4,10,11,10
645,MS,F,18,U,LE3,T,3,1,teacher,services,...,4,3,4,1,1,1,4,15,15,16
646,MS,F,18,U,GT3,T,1,1,other,other,...,1,1,1,1,1,5,6,11,12,9
647,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,6,10,10,10


In [63]:
student_df = pd.concat([stud_mat_df,stud_por_df],axis = 0)
student_df

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,MS,F,19,R,GT3,T,2,3,services,other,...,5,4,2,1,2,5,4,10,11,10
645,MS,F,18,U,LE3,T,3,1,teacher,services,...,4,3,4,1,1,1,4,15,15,16
646,MS,F,18,U,GT3,T,1,1,other,other,...,1,1,1,1,1,5,6,11,12,9
647,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,6,10,10,10


In [64]:
student_df['AlWeekly_Cons'] = student_df['Dalc']+ student_df['Walc']
student_df['Final_Grade'] = (student_df['G1'] + student_df['G2'] + student_df['G3']) //3
student_df = student_df.drop(columns = ['G1','G2','Dalc','Walc'],axis = 1)
student_df

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,health,absences,G3,AlWeekly_Cons,Final_Grade
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,no,no,4,3,4,3,6,6,2,5
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,no,5,3,3,3,4,6,2,5
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,no,4,3,2,3,10,10,5,8
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,3,2,2,5,2,15,2,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,no,no,4,3,2,5,4,10,3,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,MS,F,19,R,GT3,T,2,3,services,other,...,yes,no,5,4,2,5,4,10,3,10
645,MS,F,18,U,LE3,T,3,1,teacher,services,...,yes,no,4,3,4,1,4,16,2,15
646,MS,F,18,U,GT3,T,1,1,other,other,...,no,no,1,1,1,5,6,9,2,10
647,MS,M,17,U,LE3,T,3,1,services,services,...,yes,no,2,4,5,2,6,10,7,10


In [65]:
mn_fngrd = student_df.Final_Grade.mean()
# student_df['Success'] = student_df.apply(lambda x: 'yes' if x['G3'] >= 11 else 'no',axis = 1)
student_df['Success'] = student_df.apply(lambda x: '1' if x['Final_Grade'] >= mn_fngrd else '0',axis = 1)
student_df.head()


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,romantic,famrel,freetime,goout,health,absences,G3,AlWeekly_Cons,Final_Grade,Success
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,no,4,3,4,3,6,6,2,5,0
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,no,5,3,3,3,4,6,2,5,0
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,no,4,3,2,3,10,10,5,8,0
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,3,2,2,5,2,15,2,14,1
4,GP,F,16,U,GT3,T,3,3,other,other,...,no,4,3,2,5,4,10,3,8,0


In [66]:
student_df = pd.get_dummies(student_df, columns=['paid', 'Mjob','higher'])
student_df[['absences', 'AlWeekly_Cons', 'age', 'Medu', 'Fedu', 'freetime', 'goout','paid_no','paid_yes', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'health', 'studytime', 'higher_no', 'higher_yes', 'famrel']]

,absences,AlWeekly_Cons,age,Medu,Fedu,freetime,goout,paid_no,paid_yes,Mjob_at_home,Mjob_health,Mjob_other,Mjob_services,health,studytime,higher_no,higher_yes,famrel
0,6,2,18,4,4,3,4,1,0,1,0,0,0,3,2,0,1,4
1,4,2,17,1,1,3,3,1,0,1,0,0,0,3,2,0,1,5
2,10,5,15,1,1,3,2,0,1,1,0,0,0,3,2,0,1,4
3,2,2,15,4,2,2,2,0,1,0,1,0,0,5,3,0,1,3
4,4,3,16,3,3,3,2,0,1,0,0,1,0,5,2,0,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,4,3,19,2,3,4,2,1,0,0,0,0,1,5,3,0,1,5
645,4,2,18,3,1,3,4,1,0,0,0,0,0,1,2,0,1,4
646,6,2,18,1,1,1,1,1,0,0,0,1,0,5,2,0,1,1
647,6,7,17,3,1,4,5,1,0,0,0,0,1,2,1,0,1,2


In [40]:
# student_df['paid'] = student_df['paid'].astype('category')
# student_df['Mjob'] = student_df['Mjob'].astype('category')
# student_df['higher'] = student_df['higher'].astype('category')
# student_df = pd.get_dummies(student_df, columns=['paid', 'Mjob','higher'])

In [67]:
# excludeColumns = ()
predictors = ['absences', 'AlWeekly_Cons', 'age', 'Medu', 'Fedu', 'freetime', 'goout','paid_no','paid_yes', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'health', 'studytime', 'higher_no', 'higher_yes', 'famrel']
outcome = 'Success'
X = student_df[predictors]
y = student_df[outcome]
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.6, random_state=5)
classifier = LogisticRegression()
classifier.fit(train_X, train_y)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [68]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = classifier.predict(valid_X)
accuracy = accuracy_score(valid_y, y_pred)
classification_report = classification_report(valid_y, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{classification_report}")

#  Make Predictions for Success Outcome
success_predictions = classifier.predict(X)
print(f"success predictions: {success_predictions}")

Accuracy: 0.671451355661882
Classification Report:
              precision    recall  f1-score   support

           0       0.66      0.52      0.58       277
           1       0.68      0.79      0.73       350

    accuracy                           0.67       627
   macro avg       0.67      0.66      0.66       627
weighted avg       0.67      0.67      0.66       627

success predictions: ['1' '0' '0' ... '1' '0' '1']


In [71]:
classificationSummary(train_y, classifier.predict(train_X))
classificationSummary(valid_y, classifier.predict(valid_X))

Confusion Matrix (Accuracy 0.7122)

       Prediction
Actual   0   1
     0 114  77
     1  43 183
Confusion Matrix (Accuracy 0.6715)

       Prediction
Actual   0   1
     0 145 132
     1  74 276


In [75]:
classifier.predict_proba(valid_X)[::,1]

array([0.77945063, 0.47589105, 0.39025035, 0.50588957, 0.76509833,
       0.88741485, 0.77544206, 0.73246452, 0.70259206, 0.43042208,
       0.56093277, 0.56925741, 0.76563471, 0.45533803, 0.77809518,
       0.78250036, 0.40800283, 0.24768032, 0.74219066, 0.39159501,
       0.70158845, 0.325717  , 0.39617889, 0.80247258, 0.56443353,
       0.695526  , 0.26786195, 0.1129665 , 0.76658576, 0.31986793,
       0.32706415, 0.12258834, 0.38169265, 0.38830738, 0.17721133,
       0.12400874, 0.35804154, 0.87903629, 0.63867742, 0.80057971,
       0.76590538, 0.83210174, 0.78709231, 0.72126091, 0.67004596,
       0.48909438, 0.62438502, 0.46787699, 0.44306732, 0.35100906,
       0.79670908, 0.6952586 , 0.69472327, 0.79551497, 0.71004259,
       0.71527628, 0.84189732, 0.48903902, 0.77932724, 0.6508956 ,
       0.87627485, 0.07316724, 0.68813806, 0.65853407, 0.76457761,
       0.69252067, 0.41649938, 0.52147091, 0.68017771, 0.65826415,
       0.65950305, 0.68774108, 0.27787081, 0.3143324 , 0.88254